In [1]:
#Feature engineering for 2nd model: where products have been sold previously but then are in other shops

In [2]:
#Model definition will be:
#new products development, columns by month, columns for total sales, columns for sales per shop per month, 
#price variation in those months
#number of days sold in month

#build on products which are new, and train is also on new shops of those products
#ensure add in shops which were not sold to - ie so it correctly predicts zeros

In [3]:
#first need to source data - train, cut to new prods only... #start one month out, if new in one month prior, how builds out
    #what is my proportion I need to predict here
#when build, consider to make it generic so can replicate easily for other months and prediction



In [4]:
#start predicting October 15 (poss another version with Nov 14)
#per month, start with Sep 15, who were new products
    #then create the data set for each product for 42 shops, 
    #Target is October - look back up - where null, fill with zeros
        #for september
            #sold total amount
                #These are not yet built
                #price average
                #revenue generated in total (month or overall?)
                #increase in revenue generated per month prior
            #price var
            #sold per shop
        #for August...
#how create this in simplest reproducabile way
    #create list of new products and shop combos first
    #create month -1 ds, merge, fillna=0

In [5]:
#Read data , read firsts, id those products for 102k and how extended, 
    #create target variable - October - plus extend for missing shops
    #create for new products - will be some overlap with other score but can compare and decide which to take
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime as dt
dateCols = ['date']
train2=pd.read_csv("/home/cdsw/train_plus_not_imputted.csv",parse_dates=dateCols)
train2["TotalRevenue"]=train2.item_cnt_day*train2.item_price
train2["Month"]=train2.date.dt.month
train2["Year"]=train2.date.dt.year
train2['YM'] = train2['Year']*100+train2["Month"]
firsts=pd.read_csv("/home/cdsw/firsts.csv")
firsts.rename(columns={"YM":"FirstSale","Day":"FirstSaleD","date":"FirstSaledt"},inplace=True)
firsts15=firsts[firsts.FirstSale>=201501]
train_base=train2[train2.item_id.isin(firsts15.item_id)]

In [7]:
import itertools
#do group by and then pivot the data by YM, October is what you need
train_base["YM2"]="_"+train_base.YM.astype("str")
salesbyYM=train_base.groupby(["YM2","item_id","shop_id"])["item_cnt_day"].sum().reset_index(name="SoldPM")
#need to add in the shops where nothing is sold already

#what are the shops we need to predict for 
test=pd.read_csv("/home/cdsw/01_future-sales/data/test.csv")
test_shops=test.drop_duplicates(["shop_id"],keep="first").reset_index()
#what are the products that I have
prods=salesbyYM.drop_duplicates(subset=["item_id"],keep="first").reset_index()
#create all combinations of the shops and products
a=list(prods.item_id)
b=list(test_shops.shop_id)
c = list(itertools.product(a, b))
Combos=pd.DataFrame({"out":c})
Combos2=pd.DataFrame(Combos.out.values.tolist(),index=Combos)
Combos2.columns=["item_id","shop_id"]
Combos2["key"]=Combos2.item_id.astype("str")+"_"+Combos2.shop_id.astype("str")
salesbyYM["key"]=salesbyYM.item_id.astype("str")+"_"+salesbyYM.shop_id.astype("str")

#find the combinations which are NOT in the salesbyYM
Req=Combos2[~Combos2.key.isin(salesbyYM.key)]
#Now repeat the process for the months
a=list(set(salesbyYM.YM2))
b=list(Req.key)
c=list(itertools.product(a,b))
Adds=pd.DataFrame({"out":c})
Adds2=pd.DataFrame(Adds.out.values.tolist(),index=Adds)
Adds2.columns=["YM2","key"]
Adds2[["item_id","shop_id"]]=Adds2.key.str.split("_",expand=True)
Adds2["SoldPM"]=0

#rearrange the columns
cols = salesbyYM.columns.tolist()
Adds3=Adds2[cols]
salesbyYM2=pd.concat([salesbyYM, Adds3],axis=0)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#pivot the data so is in format to predict against 
salespivot=salesbyYM2.pivot_table(index=["item_id","shop_id"], columns="YM2",values="SoldPM", fill_value=0).reset_index()
#Target=salespivot.loc[:,["item_id", "shop_id", "_201510"]]
salespivot2=salespivot.drop(["_201511","_201512"],axis="columns")
salespivot2.columns=["item_id","shop_id", "S&P_min9","S&P_min8","S&P_min7","S&P_min6","S&P_min5",
                    "S&P_min4","S&P_min3","S&P_min2","S&P_min1","Target"]
salespivot2.head()

,item_id,shop_id,S&P_min9,S&P_min8,S&P_min7,S&P_min6,S&P_min5,S&P_min4,S&P_min3,S&P_min2,S&P_min1,Target
0,10,54,1,0,0,0,0,0,0,0,0,0
1,22,54,0,0,0,0,1,0,0,0,0,0
2,23,54,0,1,0,0,0,0,0,0,0,0
3,24,54,0,0,0,0,1,0,0,0,0,0
4,69,55,3,1,1,20,22,12,6,11,10,16


In [ ]:
##create a data set which is starts with the oldest data and adds the features - 
#having only 6 months of data where there since 12 months is bad?

In [ ]:
#add in features
    #total sales of that product - per month
    #number of shops sold in
    #price in that shop

In [32]:
#total sales for that product in prior months
Totalsales= salesbyYM2.groupby(["YM2","item_id"])["SoldPM"].sum().reset_index(name="SoldPM")
Totalsales_piv=Totalsales.pivot_table(index=["item_id"],columns="YM2", values="SoldPM", fill_value=0).reset_index()
#drop future months and target / cannot use to predict
Totalsales_piv.drop(["_201510","_201511","_201512"], axis=1,inplace=True)
Totalsales_piv.columns=["item_id", "Prod_min9","Prod_min8","Prod_min7","Prod_min6","Prod_min5",
                    "Prod_min4","Prod_min3","Prod_min2","Prod_min1"]

#Number of shops where this prod is sold
Totalshops=salesbyYM.groupby(["YM2","item_id"])["shop_id"].count().reset_index(name="CountShops")
Totalshops2=Totalshops.pivot_table(index="item_id",columns="YM2", values="CountShops",fill_value=0).reset_index()
Totalshops3=Totalshops2.drop(Totalshops2.columns.to_series()["_201510":"_201512"],axis=1)
Totalshops3.columns=["item_id", "Shop_min9","Shop_min8","Shop_min7","Shop_min6","Shop_min5",
                    "Shop_min4","Shop_min3","Shop_min2","Shop_min1"]

#how much sold in this shop previously
OverShop=salesbyYM.groupby(["YM2","shop_id"])["SoldPM"].sum().reset_index()
OverShop2=OverShop.pivot_table(index=["shop_id"],columns="YM2", values="SoldPM", fill_value=0).reset_index()
OverShop3=OverShop2.drop(OverShop2.columns.to_series()["_201510":"_201512"],axis=1)
OverShop3.columns=["shop_id", "TotShop_min9","TotShop_min8","TotShop_min7","TotShop_min6","TotShop_min5",
                    "TotShop_min4","TotShop_min3","TotShop_min2","TotShop_min1"]

#stdev of price at overall level (as not need them to apply at prod level)
StdP=train_base.groupby(["YM2","item_id"])["item_price"].std().reset_index(name="StD_Price")
StdP.StD_Price.fillna(0)
StdP2=StdP.pivot_table(index=["item_id"],columns="YM2", values="StD_Price", fill_value=0).reset_index()
StdP3=StdP2.drop(StdP2.columns.to_series()["_201510":"_201512"],axis=1)
StdP3.colums=["item_id", "PPStd_min9","PPStd_min8","PPStd_min7","PPStd_min6","PPStd_min5",
                    "PPStd_min4","PPStd_min3","PPStd_min2","PPStd_min1"]

#Other features I could build
    #price average - think std P should cover this
    #revenue generated in total (month or overall?)
    #increase in revenue generated per month prior - this could be an interesting predictor for when expand into new shops


In [33]:
#join the data
salespivot3=pd.merge(salespivot2,Totalsales_piv,how="left",on="item_id")
salespivot4=pd.merge(salespivot3,Totalshops3,how="left", on="item_id")
salespivot5=pd.merge(salespivot4, OverShop3, how="left",on="shop_id")
salespivot6=pd.merge(salespivot5, StdP3, how="left", on="item_id")

In [36]:
#write the file for modelling
salespivot6.to_csv("/home/cdsw/train_for_mod2_v1.csv")